**HAM OR SPAM CLASSIFICATION OF TEXT MESSAGES**

In [ ]:
import requests

###Importing the messsage dataset

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
                
if __name__ == "__main__":
    file_id = '1e_Azf9zGvSWsDhM9PP2sfMNKC72-iWAK'
    destination = 'data.txt'
    download_file_from_google_drive(file_id, destination)

In [ ]:
!ls

In [ ]:
with open('data.txt', 'r') as f:
  data_raw = f.readlines()

In [ ]:
def first_five_in_list(data_raw):
  l_5 = []
  if(len(data_raw)>5):
    for i in range(5):
      l_5.append(data_raw[i])
  else:
    l_5 = []    
  return l_5


first_five_in_list(data_raw)

In [ ]:
def remove_trailing_newlines(s): 
  s_clean = s.replace("\n","")
  return s_clean

In [ ]:
def mapl(f,l):
  f_l = []
  for i in range(len(l)):
    f_l.append(f(l[i]))

  return f_l

mapl(remove_trailing_newlines,data_raw) 


In [ ]:
def split_at_s(text, s):
  a = text.split(s)
  split_text = (a[0],a[1])
  return split_text

In [ ]:
def split_at_tab(text):
  b = split_at_s(text,'\t')
  return b

In [ ]:
data_clean2 = []
for line in data_clean:
  data_clean2.append(split_at_tab(line))
print(data_clean2)

In [ ]:
import string
def remove_punctuations_and_lower(text):
  return (text.translate(str.maketrans("","", string.punctuation))).lower()

In [ ]:
data_set1 = []
def rem_pun():
  for i in data_clean2:
    y = remove_punctuations_and_lower(i[1])
    data_set1.append((i[0],y))
  return data_set1
rem_pun()  


In [ ]:
count_dict = {}
def counter(l, f):
  count1 = 0
  count2 = 0
  count_dict = {}
  a = []
  a = f(l)
  for item in a:
    if(item=='ham'):
      count1 += 1
    elif(item=='spam'):
      count2 += 1
  count_dict = {'ham':count1,'spam':count2}    

  return count_dict
counter(data_set1,aux_func)

In [ ]:
  def aux_func(l):
    m = []
    for i in range(len(l)):
      m.append(l[i][0])
    return m  

In [ ]:
import random
def random_shuffle(l):
  l_shuffled = []  
  random.shuffle(l)
  for i in range(len(l)):
    l_shuffled.append(l[i])
  return l_shuffled
random_shuffle(data_set1)  

In [ ]:
import math
size = len(data_set1)
a =math.floor(len(data_set1)*0.8)
print(a)
data_train = data_set1[0:a]
data_test = data_set1[a:size]
print(len(data_test))
print(data_train)
print(data_test)

In [ ]:
vocab = []
for tuple_a in data_train:
  for words in tuple_a[1].split(" "):
    if words not in vocab:
      vocab.append(words)
print(vocab)      

In [ ]:
dict_spam = {}
dict_ham = {}
count = 0
count2 = 0
for tuple_a in data_train:
  for word in tuple_a[1].split(" "):
    if tuple_a[0] == 'ham':
      if word in dict_ham:
        dict_ham[word] += 1
        count += 1
      else:
        dict_ham[word] = 1
        count += 1
    elif tuple_a[0] == 'spam':
      if word in dict_spam:
        dict_spam[word] += 1
        count2 += 1
      else:
        dict_spam[word] = 1
        count2 += 1
print(dict_ham)
print(dict_spam)    
print(count)    

In [ ]:
dict_prob_spam = {}
dict_prob_ham = {}
a = len(vocab)
for word in vocab:
  if word in dict_ham:
    prob_ham = (dict_ham[word]+1)/(count+a)
  else:
    prob_ham = 0  
  dict_prob_ham[word] = prob_ham  
  if word in dict_spam:
    prob_spam = (dict_spam[word]+1)/(count2+a)  
  else:
    prob_spam = 0  
  dict_prob_spam[word] = prob_spam  
print(dict_prob_ham)  
print(dict_prob_spam)

In [ ]:
def predict(text, dict_prob_spam, dict_prob_ham, data_train):
  count_ham = 0
  count_spam = 0
  ham_score = 0
  spam_score = 0
  a = len(data_train)
  for tuple_a in data_train:
    if(tuple_a[0]=='ham'):
      count_ham += 1
    elif(tuple_a[0]=='spam'):
      count_spam += 1  
  p_spam = count_spam/a
  p_ham = count_ham/a
  for word in text.split(" "):
    if word in dict_prob_spam.keys():
      spam_score = p_spam * dict_prob_spam[word]
    if word in dict_prob_ham.keys():
       ham_score = p_ham * dict_prob_ham[word]
  if ham_score < spam_score:
    prediction = 'spam'
  else:
    prediction = 'ham'
  return prediction     

In [ ]:
def accuracy(data_test, dict_prob_spam, dict_prob_ham, data_train):
  accuracy = 0
  correct_pred = 0
  for tuple_a in data_test:
    prediction = predict(tuple_a[1], dict_prob_spam, dict_prob_ham, data_train)
    if tuple_a[0] == prediction:
      correct_pred +=1
  accuracy = correct_pred / len(data_test)
  return accuracy
accuracy(data_test,dict_prob_spam,dict_prob_ham,data_train)
